# Lab 13: Exercise solutions

In [1]:
with open('../course/common.py') as fin:
    exec(fin.read())

In [2]:
with open('../course/matplotlibconf.py') as fin:
    exec(fin.read())

## Exercise 1

Let's deploy an image recognition API using Tensorflow Serving. The main difference from the API we have deployed in this lab is that we will have to deal with how to pass an image to the model through tensorflow serving. Since this lab focuses on deployment, we will take a shortcut and deploy a pre-trained model that uses Imagenet. In particular, we will deploy the `Xception` model. If you are unsure about how to use a pre-trained model, please go back to [Lab 11](./11_Pretrained_models_for_images.ipynb) for a refresher.

Here are the steps you will need to complete:

- load the model in Keras
- export the model for tensorflow serving:
    - set the learning phase to zero
    - save the model with `tf.saved_model.save`
- run the model server
- write a short script that:
    - loads an image
    - pre-processes it with the appropriate function
    - serializes the image to Protobuf
    - sends the image to the server
    - receives a prediction
    - decodes the prediction with Keras `decode_prediction` function

In [3]:
import os
from os.path import join
import shutil

import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

from grpc import insecure_channel

In [4]:
from tensorflow_serving.apis.prediction_service_pb2_grpc \
    import PredictionServiceStub
from tensorflow_serving.apis.predict_pb2 \
    import PredictRequest

#### Save Xception as tensorflow model

In [5]:
model = Xception(weights='imagenet')

In [6]:
tf.keras.backend.set_learning_phase(0)

In [7]:
base_path = '/tmp/ztdl_models/xception'
sub_path = 'tfserving'
version = 1

In [8]:
export_path = join(base_path, sub_path, str(version))
shutil.rmtree(export_path, ignore_errors=True)

In [9]:
tf.saved_model.save(model, export_path)

In [10]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['predictions'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1000)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


#### Start Server

```
docker run \
    -v /tmp/ztdl_models/xception/tfserving/:/models/xception \
    -e MODEL_NAME=xception \
    -e MODEL_PATH=/models/xception \
    -p 8502:8500  \
    -p 8503:8501  \
    -t tensorflow/serving
```

#### Convert image to protobuf

In [11]:
img = image.load_img(
    './13_penguin.jpg', target_size=(299, 299))

In [12]:
img_tensor = np.expand_dims(
    image.img_to_array(img), axis=0)

In [13]:
img_scaled = preprocess_input(img_tensor)

In [14]:
data_pb = tf.compat.v1.make_tensor_proto(
    img_scaled, dtype='float', shape=img_scaled.shape)

#### Send request and retrieve response

In [15]:
channel = insecure_channel('localhost:8502')

In [16]:
stub = PredictionServiceStub(channel)

In [17]:
request = PredictRequest()

In [18]:
request.model_spec.name = 'xception'

In [19]:
request.model_spec.signature_name = 'serving_default'

In [20]:
request.inputs['input_1'].CopyFrom(data_pb)

In [21]:
result_future = stub.Predict.future(request, 5.0)

In [22]:
result = result_future.result()

#### Decode predictions

In [23]:
scores = tf.make_ndarray(result.outputs['predictions'])

In [24]:
preds = decode_predictions(scores, top=1)[0][0][1]

In [25]:
preds

'king_penguin'

## Exercise 2

The above method of serving a pre-trained model has an issue: we are doing pre-processing and prediction decoding on the client side. This is not a best practice, because it requires the client to be aware of what kind of pre-processing and decoding functions the model needs.

We want a server that takes the image as it is and returns a string with the name of the object found. 

The easy way to do this is to use the Flask app implementation we have shown in this lab and move pre-processing and decoding on the server side.

Go ahead and build a Flask version of the API that takes an image URL as a JSON string, applies pre-processing, runs and decodes the prediction and returns a string with the response.

You will not use tensorflow serving for this exercise.

Once your script is ready, save it as `13_flask_serve_xception.py`, run it as:

```
python 13_flask_serve_xception.py
```

and test the prediction with the following command:

    curl -d "http://bit.ly/2wb7uqN" \
         -H "Content-Type: application/json" \
         -X POST http://localhost:5000

If you've done things correctly, this should return:

    "king_penguin"

**Disclaimer: this script is not for production purposes. Retrieving a file from a URL is not secure, and you should avoid building an API that retrieves a file from a URL provided from the client. Here we used the URL retrieval trick to make the curl command shorter.**

In [26]:
!cat 13_flask_serve_xception.py

import os
import json
import numpy as np

from flask import Flask
from flask import request, jsonify

import tensorflow as tf
from urllib.request import urlretrieve
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

loaded_model = None

app = Flask(__name__)


def load_model():
    """
    Load model and tensorflow graph
    into global variables.
    """

    # global variables
    global loaded_model

    loaded_model = Xception(weights='imagenet')
    print("Model loaded.")

def load_image_from_url(url, target_size=(299, 299)):
    path, response = urlretrieve(url, filename='/tmp/temp_img')
    img = image.load_img(path, target_size=target_size)
    img_tensor = np.expand_dims(image.img_to_array(img), axis=0)
    return img, img_tensor

def preprocess(data):
    url = data.decode('utf-8')
